In [7]:
%%writefile src/report_1.py
import os
import requests
import json
from pymongo import MongoClient
import mylib

Client = MongoClient('mongodb://localhost:27017')
_db = Client['myDB']
_table = _db['report1']

def saveJson(_fname, _data):
    import io
    with io.open(_fname, 'a', encoding='utf-8') as json_file:
        _j = json.dump(_data, json_file, ensure_ascii=False)
        json_file.write(str(_j) + "\n")

def readJson(_fname):
    for line in open(_fname, 'r').readlines():
        _j = json.loads(line)
        print(_j['id'])

def saveDB(_data):
    _table.insert_one(_data)

def readDB():
    for tweet in _table.find():
        print(tweet['id'], tweet['text'])

def saveFile(_fname, _data):
    fp = open(_fname, 'a')
    fp.write(_data + "\n")

def doIt():
    keyPath = os.path.join(os.getcwd(), 'src', 'key.properties')
    key = mylib.getKey(keyPath)

    endpoint = 'http://openAPI.seoul.go.kr:8088'
    KEY = str(key['dataseoul'])
    TYPE = 'json'
    SERVICE = 'landActualPriceInfo'
    _startIndex = 1
    _endIndex = 50

    _maxIter = 100
    _iter = 0
    _jfname = 'src/LandActualPriceInfo.json'
    rows = []
    myJson = 0

    while _iter < _maxIter:
        url  = "/".join([endpoint, KEY, TYPE, SERVICE, str(_startIndex), str(_endIndex)])
        _json = requests.get(url).json()
        if(_iter == 0):
            print('apiSize: ', _json['landActualPriceInfo']['list_total_count'])
            myJson = _json
        if(_iter == _maxIter - 1):
            print('myDB Size: ', str(_maxIter * (_endIndex - _startIndex + 1)))
            print('last item: ', _json['landActualPriceInfo']['row'][-1])
        rows.extend(_json['landActualPriceInfo']['row'])
        #saveDB(_json)
        _startIndex += 50
        _endIndex += 50
        _iter += 1
    myJson['landActualPriceInfo']['row'] = rows
    saveJson(_jfname, myJson)

if __name__ == '__main__':
    doIt()

Overwriting src/report_1.py


In [8]:
!python src/report_1.py

apiSize:  2381895
myDB Size:  5000
last item:  {'RTMS_ID': '11110-2021-4-0006127-1', 'LAND_CD': '1111018000101260000', 'SGG_CD': '11110', 'SGG_NM': '종로구', 'BJDONG10_CD': '1111018000', 'BJDONG_NM': '교북동', 'ACC_YEAR': '2021', 'JOB_GBN': '4', 'JOB_GBN_NM': '신고', 'DEAL_YMD': '20211002', 'OBJ_SEQNO': '1', 'TOT_AREA': '43.610000', 'BLDG_AREA': '25.910000', 'RIGHT_GBN': '0', 'FLR_INFO': '2', 'BLDG_MUSE_CD': '14202', 'BLDG_MUSE_NM': '오피스텔', 'OBJ_AMT': '330000000', 'BLDG_NM': '경희궁자이', 'BUILD_YEAR': '2017'}


In [ ]:
# 파일 읽기
# 각 줄마다 json이 생성됨.
# 각 줄의 json의 row element만 저장해서 expand
# 그 결과 json으로 저장

# 다른 스크립트에서:
# json에서 
import os
import json

r = open(os.path.join('src', 'LandActualPriceInfo.json'), 'r')

